In [1]:
import os
import asyncio
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts


In [2]:
from python.agents.analyst_agent import analyst_agent

Libraries imported.
Libraries imported.


In [3]:
session_service = InMemorySessionService()
APP_NAME = "test_app"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

session = session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
# Key Concept: Runner orchestrates the agent execution loop.
runner = Runner(
    agent=analyst_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

Session created: App='test_app', User='user_1', Session='session_001'
Runner created for agent 'analyst_agent'.


In [4]:
def call_agent(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

    for event in events:
        if event.get_function_calls:
            calls = event.get_function_calls()
            if calls:
                for call in calls:
                    tool_name = call.name
                    arguments = call.args # This is usually a dictionary
                    print(f"  Tool: {tool_name}, Args: {arguments}")
                    # Application might dispatch execution based on this
    
        if event.is_final_response():
            final_response = event.content.parts[0].text
            # print(final_response)
            return final_response

In [ ]:
query = "จากข่าวเศรษฐกิจล่าสุด หุ้นตัวไหนได้ประโยชน์ และ หุ้นตัวไหนเสียประโยชน์"
r_1 = call_agent(query)

  Tool: transfer_to_agent, Args: {'agent_name': 'search_agent'}
  Tool: web_grounding, Args: {'query': 'ข่าวเศรษฐกิจล่าสุดประจำสัปดาห์นี้ (29 เมษายน - 6 พฤษภาคม 2568)'}
  Tool: web_grounding, Args: {'query': 'ผลกระทบของข่าวเศรษฐกิจต่อตลาดหุ้นไทย สัปดาห์นี้ (29 เมษายน - 6 พฤษภาคม 2568)'}
  Tool: web_grounding, Args: {'query': 'หุ้นที่คาดว่าจะได้ประโยชน์จากข่าวเศรษฐกิจล่าสุด (29 เมษายน - 6 พฤษภาคม 2568)'}
  Tool: web_grounding, Args: {'query': 'หุ้นที่คาดว่าจะเสียประโยชน์จากข่าวเศรษฐกิจล่าสุด (29 เมษายน - 6 พฤษภาคม 2568)'}
  Tool: web_grounding, Args: {'query': 'บทวิเคราะห์ตลาดหุ้นไทยล่าสุดประจำสัปดาห์นี้ (29 เมษายน - 6 พฤษภาคม 2568)'}
  Tool: crawl, Args: {'url': 'https://www.krungsri.com/th/research/macroeconomic/weekly/20250506'}
  Tool: crawl, Args: {'url': 'https://www.bangkokbiznews.com/finance/stock/1154343'}
  Tool: crawl, Args: {'url': 'https://www.kaohoon.com/news/685700'}
[INIT].... → Crawl4AI 0.6.2
[FETCH]... ↓ https://www.krungsri.com/th/research/macroeconomic/weekly/2025050

In [ ]:
print(f"Response: {r_1}")

Response: จากการรวบรวมข้อมูลและวิเคราะห์จากแหล่งต่างๆ สรุปได้ดังนี้:

**ภาพรวมเศรษฐกิจและตลาดหุ้น:**

*   **ความเสี่ยงขาลงของเศรษฐกิจไทยเพิ่มขึ้น:** หลายสถาบันปรับลดคาดการณ์ GDP ปี 2568 ลงใกล้ระดับ 2% กนง. ปรับลดดอกเบี้ยนโยบายต่อเนื่องสู่ระดับ 1.75% และมีสัญญาณอาจผ่อนคลายต่อเนื่องขึ้นอยู่กับความคืบหน้าของการเจรจาการค้า
*   **SET Index ผันผวน:** ตลาดหุ้นไทยยังคงมีความผันผวนและถูกกดดันจากปัจจัยภายนอก เช่น นโยบายการค้าของสหรัฐฯ
*   **เน้นลงทุนรายตัว:** สำนักวิจัยทิสโก้แนะนำให้พิจารณาลงทุนในหุ้นรายตัวมากกว่าดูที่ระดับ SET Index เนื่องจากภาพรวมตลาดอาจบิดเบือนจากความเป็นจริง

**หุ้นที่คาดว่าจะได้ประโยชน์:**

*   **หุ้นที่ได้ประโยชน์จากบาทอ่อนค่า:**
*   **หุ้นพื้นฐานดีขนาดใหญ่:** ที่คาดว่าจะเป็นเป้าเม็ดเงินกองทุนลดหย่อนภาษี (TESG) ไหลเข้าปลายปี
*   **กลุ่มนิคมอุตสาหกรรม:** AMATA และ WHA (หากแนวโน้มการกีดกันทางการค้าเพิ่มสูงขึ้น)
*    **กลุ่มเดินเรือ** TTA, PSL, RCL
*   **กลุ่มพลังงาน** PTTEP, BANPU
*   **กลุ่มโรงพยาบาล** BDMS

**หุ้นที่คาดว่าจะเสียประโยชน์:**

*   **กลุ่มส่งออก**
*   **กลุ่มช

In [ ]:
query = "อธิบายเหตุผลที่หุ้นแต่ละกลุ่มนี้ได้ประโยชน์ และเสียประโยชน์จากข่าวเศรษฐกิจล่าสุด"
r_2 = call_agent(query)


In [ ]:
print(f"Response: {r_2}")

Response: จากข้อมูลที่ได้รวบรวมมา สามารถอธิบายเหตุผลที่หุ้นแต่ละกลุ่มได้ประโยชน์และเสียประโยชน์จากข่าวเศรษฐกิจล่าสุด (อัปเดต ณ วันที่ 6 พฤษภาคม 2568) ได้ดังนี้:

**หุ้นที่คาดว่าจะได้ประโยชน์:**

*   **หุ้นที่ได้ประโยชน์จากบาทอ่อนค่า:**
    *   **เหตุผล:** ค่าเงินบาทที่อ่อนค่าทำให้สินค้าส่งออกของไทยมีราคาถูกลงในตลาดต่างประเทศ ทำให้มีความต้องการสินค้าไทยมากขึ้น ส่งผลดีต่อบริษัทที่เน้นการส่งออก
*   **หุ้นพื้นฐานดีขนาดใหญ่ ที่คาดว่าจะเป็นเป้าเม็ดเงินกองทุนลดหย่อนภาษี (TESG) ไหลเข้าปลายปี:**
    *   **เหตุผล:** กองทุน TESG มีนโยบายลงทุนในหุ้นที่มีปัจจัยพื้นฐานดี และมีขนาดใหญ่ ซึ่งเป็นที่สนใจของนักลงทุนที่ต้องการลดหย่อนภาษี ทำให้หุ้นกลุ่มนี้มีความต้องการสูงขึ้นในช่วงปลายปี
*   **กลุ่มนิคมอุตสาหกรรม (AMATA และ WHA):**
    *   **เหตุผล:** หากแนวโน้มการกีดกันทางการค้าเพิ่มสูงขึ้น บริษัทต่างชาติอาจย้ายฐานการผลิตมายังประเทศไทย ซึ่งนิคมอุตสาหกรรมจะเป็นแหล่งรองรับการลงทุนเหล่านี้
*   **กลุ่มเดินเรือ (TTA, PSL, RCL):**
    *   **เหตุผล:** หากเศรษฐกิจโลกฟื้นตัว ความต้องการขนส่งสินค้าทางทะเลจะเพิ่มขึ้

In [13]:
query = "ตัวไหนในหุ้นที่ได้ประโยชน์มากที่สุด มีปันผลสูงสุด และ ราคาถูกสุด เลือกมา 1 ตัว พร้อมเหตุผล"
r_3 = call_agent(query)


In [14]:
print(f"Response: {r_3}")

Response: จากการวิเคราะห์ข้อมูล ณ วันที่ 6 พฤษภาคม 2568 (โดยมีข้อจำกัดเรื่องข้อมูลที่จำกัดและความคลาดเคลื่อน) ผมขอเลือก **WHA (บริษัท ดับบลิวเอชเอ คอร์ปอเรชั่น จำกัด (มหาชน))** เป็นหุ้นที่น่าสนใจที่สุดในกลุ่มหุ้นที่คาดว่าจะได้ประโยชน์ โดยมีเหตุผลดังนี้:

1.  **ราคาค่อนข้างถูก:** ราคาหุ้น WHA อยู่ในช่วง 2.82 - 3.16 บาท ซึ่งถือว่าค่อนข้างต่ำเมื่อเทียบกับ AMATA (14.00 - 18.80 บาท) และ RCL (22.70 บาท) ทำให้มีโอกาสในการเข้าลงทุนได้ง่ายกว่า และมีโอกาสในการทำกำไรจากส่วนต่างราคา (Capital Gain) ได้มากกว่าหากราคาหุ้นปรับตัวสูงขึ้น
2.  **มีปันผล:** แม้อัตราปันผลล่าสุดของ WHA (0.1170 บาท/หุ้น ในปี 2567) อาจจะไม่สูงที่สุด แต่ก็แสดงให้เห็นว่าบริษัทมีการจ่ายปันผลอย่างต่อเนื่อง ซึ่งเป็นผลดีต่อนักลงทุนที่ต้องการรายได้จากเงินปันผล
3.  **แนวโน้มการเติบโต:** WHA ได้รับประโยชน์จากการย้ายฐานการผลิตของนักลงทุนต่างชาติ ซึ่งเป็นผลมาจากนโยบายกีดกันทางการค้าของสหรัฐฯ ทำให้นิคมอุตสาหกรรมของ WHA มีโอกาสในการเติบโตของรายได้และกำไร
4.  **กลุ่มนิคมอุตสาหกรรมยังคงน่าสนใจ:** แม้ว่าจะมีข่าวหุ้นกลุ่มนิคมอุตสาหกรรมปรับตัว